Before you turn this problem set in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All). Your code should run from top to bottom with no errors. Failure to do this will result in loss of points.

You should not use `install.packages()` anywhere. You may assume that we have already installed all the packages needed to run your code.

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE" and delete the `stop()` functions, as well as your name and collaborators below:

In [94]:
NAME = "aayushap"  # your uniqname 
COLLABORATORS = c("pjmerica", "sohumm", "kaspersj")  # vector of uniqnames of your collaborators, if any
## IMPORTANT: you must also have set your group on Canvas. This is only used as a backup.

---

In [95]:
library(tidyverse)
library(stringr)
library(lubridate)

# STATS 306
## Problem Set 8: Functions
Each problem is worth two to four points, depending on difficulty, for a total of 20.

*Note*: you do not need to use `install.packages()` in this notebook. You may assume that we have already installed all of the necessary packages when we run your code.

#### Problem 1 (2 pts.)
Create a function `n_entries(v, x)` which, given a vector `v` and a scalar `x`, returns the number of times that `x` occurs in `v`.

In [96]:
n_entries = function (v,x) {
if(x %in% v) {
    return(sum(x == v))
  }
}


In [97]:
stopifnot(n_entries(1:10, 1) == 1)
stopifnot(n_entries(c("a", "a", "b", "c"), "a") == 2)
stopifnot(n_entries(c(), 0) == 0)

#### Problem 2 (3 pts.)
Create a function `days_between(v)` which, given a vector `v` of $n$ dates, sorts `v` in ascending order and then returns a new vector of length $n-1$ of the number of days which elapsed between each successive date. Additionally, each entry in the returned vector should have a name of the form `mm/dd/yyyy to mm/dd/yyyy` indicating the corresponding date span.
For example:
```{r}
> days_between(c("Jan. 20, 2009", "July 20, 1969", "June 28, 1914", "July 14, 1789", "Oct. 14, 1066"))
Time differences in days
10/14/1066 to 07/14/1789 07/14/1789 to 06/28/1914 06/28/1914 to 07/20/1969 
                  263979                    45638                    20111 
07/20/1969 to 01/20/2009 01/20/2009 to 01/01/2011 
                   14429                      711
```
(You may assume that all dates that will be inputted to `days_between(v)` are in the format given in the example, and between January 1, 1000 and December 31, 9999).

In [98]:
 days_between = function(v){
    v = sort(mdy(v))
    n = length(v)
    vec1 = v[2:n]
    vec2 = v[1:n-1]
    vec_diff = vec1 - vec2
    vec1 = format(vec1, format = "%m/%d/%Y")
    vec2 = format(vec2, format = "%m/%d/%Y")
    labels = str_c(vec2, vec1, sep = " to ")
    names(vec_diff) = labels
    return(vec_diff)
}

In [99]:
db = days_between(c("Jan. 20, 2009", "July 20, 1969", "June 28, 1914", "July 14, 1789", 
                    "Oct. 14, 1066", "January 1, 2011"))
stopifnot(all(db == c(263979, 45638, 20111, 14429, 711)))
stopifnot(all(names(db) == c("10/14/1066 to 07/14/1789",
                             "07/14/1789 to 06/28/1914",
                             "06/28/1914 to 07/20/1969",
                             "07/20/1969 to 01/20/2009",
                             "01/20/2009 to 01/01/2011")))

#### Problem 3 (3 pts.)
Create a function `standardize(df)` which, given a tibble `df` with numerical columns, returns a new data frame where each column is standardized (has mean zero and variance one, accomplished by subtracting the column mean and dividing by its standard deviation). For example:
```{r}
> (df <- tibble(x=1:3, y=4:6, z=c(0, 0, 1)))
  x y z
1 1 4 0
2 2 5 0
3 3 6 1
> standardize(df)
  x  y  z         
1 -1 -1 -0.5773503
2  0  0 -0.5773503
3  1  1  1.1547005
```

In [100]:
standardize = function(df) {
    mutate_all(df, scale)
}

In [101]:
stopifnot(all(
    map2_lgl(
         standardize(tibble(x=1:3, y=4:6, z=7:9)),
         tibble(x=c(-1, 0, 1), y=c(-1, 0, 1), z=c(-1, 0, 1)),
         ~ all(near(.x, .y))
         )
    )
)

#### Problem 4 (2 pts.)
Create a function `str_rev(s)` which, given a string `s`, returns the reversed string. For example, 
```{r}
> str_rev("Hello my name is Jonathan")
[1] "!nahtanoJ si eman ym olleH"
```

In [102]:
str_rev = function(s) {
     sapply(lapply(strsplit(s, NULL), rev), paste, collapse = "")
}

In [103]:
stopifnot(str_rev("Hello my name is Jonathan!") == "!nahtanoJ si eman ym olleH")
stopifnot(str_rev(str_rev("anything")) == "anything")

#### Problem 5 (2 pts.)
Thanksgiving occurs on the fourth Thursday in November. Write a function `thanksgiving_day(year)` which, given a year, returns the date of Thanksgiving for that year:

(There exist R libraries that compute this for you. You may use them to test your code. However, the solution itself  should rely on the base R / lubridate date manipulation commands only.)

In [104]:
thanksgiving_day <- function(year) {
    date = as.Date(ISOdate(year, 11, 1))
    date1 = date + dweeks(3)
    weekday = wday(date1)
   
 while(weekday != 5) {
     date1 = date1 + ddays(1)
     weekday = wday(date1)
 }
    return(date1)
}


In [105]:
stopifnot(exists("thanksgiving_day"))
stopifnot(thanksgiving_day(2018) == make_datetime(2018, 11, 22))

#### Problem 6 (2 pts.)
A *recursive* function is a function that calls itself. For example, the following recursive function computes the $k$th factorial number, $$k! := \prod_{i=1}^k i,\quad k \ge 0.$$

In [106]:
factorial = function(k) {
    if (k <= 0) {
        return(1)
    }
    k * factorial(k - 1)
}
factorial(5)

[1] 120

Write a recursive function `fib(k)` that computes $F_k$, the $k$th Fibonacci number. (The Fibonacci sequence is defined by the recurrence $F_{k+2} = F_{k+1} + F_{k}$ for $k\ge 2$, with $F_1 = F_2 = 1$.)

In [107]:
fib = function(k) {
 if(k <= 1) {
return(k)
    } 
 else {
    return(fib(k-1) + fib(k-2))
        }
            }


In [108]:
stopifnot(all(sapply(1:10, fib) == c(1,1,2,3,5,8,13,21,34,55)))

#### Problem 7 (2 pts.)
In Calculus II you learned that $$\int_0^1 f(x)\,\mathrm{d} x = \lim_{n\to\infty} \frac{1}{n} \sum_{i=1}^n f(i/n).$$ The summation on the right-hand side is called a *Riemann sum*. Setting $n$ to a large (but finite) number gives an algorithm for numerically computing the definite integral on the left-hand side.

Write a function `int01(f)` that takes a single argument `f`, itself a function, and uses this formula to approximate the integral of `f` over the interval $[0,1]$. (Choose $n=1000$ subdivisions in order to compute the sum.)

In [109]:
int01 = function(f) {
    1/1000 * sum(f(1:1000/1000))
   }

In [110]:
stopifnot(near(int01(function(x) rep(1, length(x))), 1))
stopifnot(near(int01(function(x) x), 1/2, tol = 1e-3))                     
stopifnot(near(int01(function(x) sin(2 * pi * x)), 0))

#### Problem 8 (4 pts.)
You may have noticed that `int01` is fairly inaccurate. For example, while $\int_0^1 x^2\,\mathrm{d}x = 1/3$, we see that 
```{r}
> int01(function(x) x^2)
[1] 0.3338335
```
is only correct to three decimal places. This is because `int01` does not take the shape of the function `f` into account. To obtain more accuracy, we can use [Simpson's rule](https://en.wikipedia.org/wiki/Simpson%27s_rule):
$$\int_0^1 f(x)\,\mathrm{d}x \approx \frac{1}{6n} \sum_{i=1}^n f\left(\frac{i-1}{n}\right) + 4f\left(\frac{i - 1/2}{n}\right)
+ f\left(\frac{i}{n}\right),$$ for large $n$.

Implement Simpson's rule (again using $n=1000$) in a function called `simps01(f)`, and use it to define a function `rel_acc(f, truth)` which compares the relative accuracy of the Simpson's rule against `int01`. The function should take two arguments, the function to be integrated `f` and $$\texttt{truth} = \int_0^1 f(x)\,\mathrm{d}x,$$
the true value of the integral. It should return the relative accuracy, defined as $$\texttt{relacc(f, truth)} = \frac{|\texttt{simps01(f)} - \texttt{truth}|}{|\texttt{int01(f)}-\texttt{truth}|}.$$ (Hence, small values of `rel_acc` indicate that Simpson's rule is more accurate.)

In [111]:
i = 1:1000

simps01 = function(f) {
    (1/(6*1000)) * sum(f((i-1)/1000) + 4*f((i-.5) / 1000) + f(i/1000))
}

rel_acc = function(f, truth) {
    abs(simps01(f) - truth) / abs(int01(f) - truth)
}

In [112]:
stopifnot(rel_acc(function(x) x^2, 1/3) == 0)
stopifnot(rel_acc(function(x) x^3, 1/4) == 0)
stopifnot(near(rel_acc(sin, 1 - cos(1)), 5.7e-12))